In [1]:
from plotly import express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
import cv2

In [3]:
from epi.plotlyx.express import scatter_3d
from epi.plotlyx.utils.fig import np_to_plotly
from epi.plotlyx.render import CameraCoordinateRenderer, render_camera_axes, render_axes

In [4]:
from epi import geometry as geom

In [5]:
from typing import Union, List, Tuple

In [6]:
import numpy as np
from IPython.core.debugger import set_trace

In [7]:
from epi.camera import ProjCamera
from epi.model import read_vertices, Model

In [8]:
PLOTLY_UP = dict(x=0, y=1, z=0)
PLOTLY_CENTER = dict(x=0, y=0, z=0)
PLOTLY_EYE = dict(x=0, y=0, z=2)
SCENE_SCALE = 16

In [9]:
vertices = read_vertices("./building_04.obj")
house_model = Model(
    vertices,
    color=np.linalg.norm(vertices, axis=-1),
    position=np.array([0, 0, 10]),
)

In [10]:
proj_camera1 = ProjCamera.from_gaze(
    house_model.center,
    np.array([-1, 0, 15]),
    0.2,
    640,
    480,
    xpixel_mm=0.001,
    ypixel_mm=0.001,
)

proj_camera2 = ProjCamera.from_gaze(
    house_model.center,
    np.array([2, 0, 15]),
    0.2,
    640,
    480,
    xpixel_mm=0.001,
    ypixel_mm=0.001,
)

In [ ]:
fig = go.Figure(
    data=[
        scatter_3d(
            house_model.vertices,
            marker=dict(color=house_model.color, symbol="x", size=1),
        )
    ],
)

fig.update_layout(
    scene=dict(
        camera=dict(
            up=PLOTLY_UP, eye=np_to_plotly(proj_camera1.gaze)
        ),
        xaxis=dict(
            nticks=20,
            range=[-SCENE_SCALE, SCENE_SCALE],
        ),
        yaxis=dict(
            nticks=20,
            range=[-SCENE_SCALE, SCENE_SCALE],
        ),
        zaxis=dict(
            nticks=20,
            range=[-SCENE_SCALE, SCENE_SCALE],
        ),
    ),
    width=600,
    height=600,
    showlegend=False,
    margin=dict(r=20, l=10, b=10, t=10),
)


render_camera_axes(fig, proj_camera2)
render_camera_axes(fig, proj_camera1)
fig.update_scenes(aspectmode="data")

fig

In [11]:
fig = go.Figure(
    layout=dict(
        scene=dict(
            camera=dict(
                up=PLOTLY_UP,
            ),
            xaxis=dict(
                nticks=20,
                range=[SCENE_SCALE, -SCENE_SCALE],
            ),
            yaxis=dict(
                nticks=20,
                range=[-SCENE_SCALE, SCENE_SCALE],
            ),
            zaxis=dict(
                nticks=20,
                range=[-SCENE_SCALE, SCENE_SCALE],
            ),
        ),
        width=600,
        height=600,
        showlegend=False,
        margin=dict(r=20, l=10, b=10, t=10),
    )
)

In [12]:
camera1_renderer = CameraCoordinateRenderer(proj_camera1, fig=fig)
camera2_renderer = CameraCoordinateRenderer(proj_camera2, fig=fig)

In [ ]:
camera1_renderer.render(house_model)

In [ ]:
camera2_renderer.render(house_model)

In [13]:
img1, depth1, world_idx1 = proj_camera1.render_img(house_model.vertices, color=1)
img2, depth2, world_idx2 = proj_camera2.render_img(house_model.vertices, color=1)

In [14]:
from epi.image import normalize, ops as img_ops

In [15]:
depth2_3c = img_ops.to_3channels(255 * normalize.min_max(depth2))
depth1_3c = img_ops.to_3channels(255 * normalize.min_max(depth1))

In [ ]:
fig = make_subplots(1, 2)
fig.add_trace(go.Image(z=depth1_3c), 1, 1)
fig.add_trace(go.Image(z=depth2_3c), 1, 2)
fig

In [16]:
in_img1_idxs = set(world_idx1[np.where(~np.isnan(world_idx1))].astype(np.int32))
in_img2_idxs = set(world_idx2[np.where(~np.isnan(world_idx2))].astype(np.int32))

In [17]:
epipolar_indices = np.array(list(in_img1_idxs.intersection(in_img2_idxs)))
epipolar_vertices = house_model.vertices[epipolar_indices]
epipolar_color = house_model.color[epipolar_indices]

In [18]:
x_min, y_min, z_min = np.min(epipolar_vertices, axis=0)
x_max, y_max, z_max = np.max(epipolar_vertices, axis=0)

In [19]:
st_kps = proj_camera1.project_vertices(epipolar_vertices, drop_last=True)
nd_kps = proj_camera2.project_vertices(epipolar_vertices, drop_last=True)

F, mask = cv2.findFundamentalMat(
    np.int32(st_kps),
    np.int32(nd_kps),
    
)
st_kps = st_kps[mask.flatten().astype(np.bool_)]
nd_kps = nd_kps[mask.flatten().astype(np.bool_)]

In [20]:
lines1 = cv2.computeCorrespondEpilines(np.int32(nd_kps).reshape(-1,1,2), 2,F)
lines2 = cv2.computeCorrespondEpilines(np.int32(st_kps).reshape(-1,1,2), 1,F)

In [21]:
def compute_y_at_x(x, line):
    return (-line[2] - line[0] * x) / line[1]

In [22]:
lines1 = cv2.computeCorrespondEpilines(nd_kps.reshape(-1,1,2), 2,F)
lines2 = cv2.computeCorrespondEpilines(st_kps.reshape(-1,1,2), 1,F)
lines1 = lines1.reshape(-1, 3)
lines2 = lines2.reshape(-1, 3)

In [23]:
nd_lines = (F @ (geom.to_homogenous(st_kps, axis=1)).T).T
st_lines = (F.T @ (geom.to_homogenous(nd_kps, axis=1)).T).T

In [24]:
def make_lines(lines: np.ndarray, points: np.ndarray, xlim: Tuple[int, int], **kwargs):
    xmin, xmax = xlim
    go_lines = [
        go.Scatter(
            x=[xmin, xmax],
            y=[
                compute_y_at_x(xmin, line),
                compute_y_at_x(xmax, line),
            ],
            mode="lines",
        )
        for line in lines
    ]
    go_lines.append(
        go.Scatter(
            x=points[:, 0],
            y=points[:, 1],
            mode="markers",
            **kwargs
        )

    )
    return go_lines

In [25]:
import plotly.subplots as ps

In [ ]:
no_of_points = 100

fig = ps.make_subplots(
    rows=2,
    cols=2,
    row_heights=[400, 400],
    column_widths=[600, 600],
)

fig.update_layout(
    showlegend=False,
)

for ba in make_lines(lines1[:no_of_points], st_kps[:no_of_points], xlim=[0, 640]):
    fig.add_trace(ba, col=1, row=1)

for ba in make_lines(lines2[:no_of_points], nd_kps[:no_of_points], xlim=[0, 640]):
    fig.add_trace(ba, col=2, row=1)

for ba in make_lines(st_lines[:no_of_points], st_kps[:no_of_points], xlim=[0, 640]):
    fig.add_trace(ba, col=1, row=2)

for ba in make_lines(nd_lines[:no_of_points], nd_kps[:no_of_points], xlim=[0, 640]):
    fig.add_trace(ba, col=2, row=2)

fig.add_trace(
    go.Scatter(
        x=st_kps[:, 0], y=st_kps[:, 1], mode="markers", marker=dict(size=1)
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=nd_kps[:, 0], y=nd_kps[:, 1], mode="markers", marker=dict(size=1)
    ),
    row=1,
    col=2,
)

fig.add_trace(
    go.Scatter(
        x=st_kps[:, 0], y=st_kps[:, 1], mode="markers", marker=dict(size=1)
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=nd_kps[:, 0], y=nd_kps[:, 1], mode="markers", marker=dict(size=1)
    ),
    row=2,
    col=2,
)

fig

In [26]:
kp1_ncc = geom.from_homogenous(geom.from_homogenous(proj_camera1.view @ geom.to_homogenous(epipolar_vertices.T))).T
kp2_ncc = geom.from_homogenous(geom.from_homogenous(proj_camera2.view @ geom.to_homogenous(epipolar_vertices.T))).T

In [27]:
E = proj_camera2.K.T @ F @ proj_camera1.K

In [28]:
Ep, e_mask = cv2.findEssentialMat(kp1_ncc, kp2_ncc)
kp1_ncc = kp1_ncc[e_mask.flatten().astype(np.bool_)]
kp2_ncc = kp2_ncc[e_mask.flatten().astype(np.bool_)]

In [29]:
def compute_lines(
    mat: np.ndarray,
    points: np.ndarray,
    img_num=1,
) -> np.ndarray:
    if img_num == 1:
        return (mat.T @ geom.to_homogenous(points.T)).T
    elif img_num == 2:
        return (mat @ geom.to_homogenous(points.T)).T
    else: 
        raise ValueError(f"`img_num` should be either 1 or 2, not {img_num} ")
    

In [30]:
lines2_ep_ncc = compute_lines(Ep, kp1_ncc, 2)
lines1_ep_ncc = compute_lines(Ep, kp2_ncc, 1)

lines2_ncc = compute_lines(E, kp1_ncc, 2)
lines1_ncc = compute_lines(E, kp2_ncc, 1)

In [31]:
min_x1_ncc, min_y1_ncc = kp1_ncc.min(axis=0)
max_x1_ncc, max_y1_ncc = kp1_ncc.max(axis=0)

min_x2_ncc, min_y2_ncc = kp2_ncc.min(axis=0)
max_x2_ncc, max_y2_ncc = kp2_ncc.max(axis=0)

In [ ]:
no_of_points = 100

fig = ps.make_subplots(
    rows=2,
    cols=2,
    row_heights=[400, 400],
    column_widths=[600, 600],
)

fig.update_layout(
    showlegend=False,
)

for ba in make_lines(
    lines1_ep_ncc[:no_of_points],
    kp1_ncc[:no_of_points],
    xlim=[min_x1_ncc, max_x1_ncc],
):
    fig.add_trace(ba, col=1, row=1)

for ba in make_lines(
    lines2_ep_ncc[:no_of_points],
    kp2_ncc[:no_of_points],
    xlim=[min_x2_ncc, max_x2_ncc],
):
    fig.add_trace(ba, col=2, row=1)


for ba in make_lines(
    lines1_ncc[:no_of_points],
    kp1_ncc[:no_of_points],
    xlim=[min_x1_ncc, max_x1_ncc],
):
    fig.add_trace(ba, col=1, row=2)

for ba in make_lines(
    lines2_ncc[:no_of_points],
    kp2_ncc[:no_of_points],
    xlim=[min_x2_ncc, max_x2_ncc],
):
    fig.add_trace(ba, col=2, row=2)


fig.add_trace(
    go.Scatter(
        x=kp1_ncc[:, 0],
        y=kp1_ncc[:, 1],
        mode="markers",
        marker=dict(size=1),
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=kp2_ncc[:, 0],
        y=kp2_ncc[:, 1],
        mode="markers",
        marker=dict(size=1),
    ),
    row=1,
    col=2,
)

fig.add_trace(
    go.Scatter(
        x=kp1_ncc[:, 0],
        y=kp1_ncc[:, 1],
        mode="markers",
        marker=dict(size=1),
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=kp2_ncc[:, 0],
        y=kp2_ncc[:, 1],
        mode="markers",
        marker=dict(size=1),
    ),
    row=2,
    col=2,
)

fig

In [32]:
v = np.zeros((4, 4))

In [33]:
def view_from_Rt(R: np.ndarray, t: np.ndarray) -> np.ndarray:
    Rt3x4 = np.concatenate([R, t], axis=1)
    Rt4x4 = np.concatenate([Rt3x4, np.array([[0, 0, 0, 1]])])
    return Rt4x4

In [34]:
cam1 = view_from_Rt(np.eye(3), np.zeros((3, 1)))

In [35]:
R1, R2, t = cv2.decomposeEssentialMat(E)

In [36]:
view1 = view_from_Rt(R1, t)
view2 = view_from_Rt(R2, t)
view3 = view_from_Rt(R1, -t)
view4 = view_from_Rt(R2, -t)

In [37]:
da1 = geom.from_homogenous((np.linalg.inv(view1) @ geom.to_homogenous(geom.to_homogenous(kp2_ncc, axis=1), axis=1).T)).T
da2 = geom.from_homogenous((np.linalg.inv(view2) @ geom.to_homogenous(geom.to_homogenous(kp2_ncc, axis=1), axis=1).T)).T
da3 = geom.from_homogenous((np.linalg.inv(view3) @ geom.to_homogenous(geom.to_homogenous(kp2_ncc, axis=1), axis=1).T)).T
da4 = geom.from_homogenous((np.linalg.inv(view4) @ geom.to_homogenous(geom.to_homogenous(kp2_ncc, axis=1), axis=1).T)).T

In [38]:
pos = geom.from_homogenous(np.linalg.inv(view4) @ np.array([0, 0, 0, 1]))

In [39]:
f = np.linalg.inv(R2) @ np.array([0, 0, 1])
r = np.linalg.inv(R2) @ np.array([1, 0, 0])
u = np.linalg.inv(R2) @ np.array([0, 1, 0])

In [40]:
cam2_recovered = ProjCamera(
    pos,
    f,
    proj_camera2.focal_length,
    proj_camera2.width,
    proj_camera2.height,
    proj_camera2.xpixel_mm,
    proj_camera2.ypixel_mm,
    right=r,
    up=u,
)

cam1_recovered = proj_camera1.copy_with_basis()

In [ ]:
ammount = -1
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=kp1_ncc[:ammount, 0],
            y=kp1_ncc[:ammount, 1],
            z=np.ones_like(kp1_ncc[:ammount, 1]),
            name="st_cam",
            mode="markers",
            marker=dict(
                symbol="x",
                size=1,
            ),
        ),
        go.Scatter3d(
            x=da4[:ammount, 0],
            y=da4[:ammount, 1],
            z=da4[:ammount, 2],
            name="nd_cam",
            mode="markers",
            marker=dict(
                symbol="x",
                size=1,
            ),
        ),
    ],
    layout=dict(
        scene=dict(
            camera=dict(
                up=PLOTLY_UP,
            ),
            xaxis=dict(
                nticks=20,
                range=[-SCENE_SCALE, SCENE_SCALE],
            ),
            yaxis=dict(
                nticks=20,
                range=[-SCENE_SCALE, SCENE_SCALE],
            ),
            zaxis=dict(
                nticks=20,
                range=[SCENE_SCALE, -SCENE_SCALE],
            ),
        )
    ),
)
render_camera_axes(fig, cam1_recovered, 1, f"rgb(0, 0, 255)")
render_camera_axes(fig, cam2_recovered, 1, f"rgb(255, 0, 0)")
fig

In [41]:
from epi.plotlyx.utils.color import nps_to_rgbs

In [42]:
from epi.plotlyx.render import render_camera_axes

In [63]:
v1 = np.random.uniform(0, 10, (20 ,3))
v2 = np.random.uniform(0, 10, (20, 3))

In [114]:
from epi.utils.index import Slice
from epi.vec import to_origin_direction

In [107]:
from epi import triangulation 
def to_posdir(first_origin, first_points, second_origin, seconf_points):
    first_origin = np.broadcast_to(first_origin, first_points.shape)
    first_directions = geom.normalize(first_points - first_origin)

    second_origin = np.broadcast_to(second_origin, seconf_points.shape)
    second_directions = geom.normalize(seconf_points - second_origin)

    return 

In [112]:
ammount = -1
slicee = Slice[:ammount]
alpha = 5
beta = 6

# colors = np.random.randint(0, 255, (ammount, 3))
# rgbs = nps_to_rgbs(colors)

fig = go.Figure(
    layout=dict(
        scene=dict(
            camera=dict(
                up=PLOTLY_UP,
            ),
            xaxis=dict(
                nticks=20,
                range=[-SCENE_SCALE, SCENE_SCALE],
            ),
            yaxis=dict(
                nticks=20,
                range=[-SCENE_SCALE, SCENE_SCALE],
            ),
            zaxis=dict(
                nticks=20,
                range=[SCENE_SCALE, -SCENE_SCALE],
            ),
        )
    ),
)
# for (x, y, z), rgb in zip(alpha * geom.add_col(kp1_ncc[slicee], 1), rgbs):
#     fig.add_trace(
#         go.Scatter3d(
#             x=[0, x], y=[0, y], z=[0, z], mode="lines", marker=dict(color=rgb)
#         ),
#     )
#
# for (x, y, z), rgb in zip(beta * (da4[slicee] - pos), rgbs):
#     fig.add_trace(
#         go.Scatter3d(
#             x=[pos[0], pos[0] + x],
#             y=[pos[1], pos[1] + y],
#             z=[pos[2], pos[2] + z],
#             mode="lines",
#             marker=dict(color=rgb),
#         ),
#     )

render_camera_axes(fig, cam2_recovered)
render_camera_axes(fig, cam1_recovered)
first_origins, first_directions = to_origin_direction(
    [0, 0, 0],
    geom.add_col(kp1_ncc[slicee], 1),
)
second_origins, second_directions = to_origin_direction(
    pos,
    da4[slicee],
)
midpoints = triangulation.mid_point(
    first_origins, first_directions, second_origins, second_directions
)

fig.add_trace(
    go.Scatter3d(
        x=midpoints[:, 0],
        y=midpoints[:, 1],
        z=midpoints[:, 2],
        mode="markers",
        marker=dict(size=1, symbol="x", color="rgb(0, 0, 255)"),
    ),
)
fig

1. add axis to functions for homogenuous transformations  Done
2. Add visualization for points on the epipolar lines Done
3. Compute essential matrix Done
4. Use essential matrix to compute camera matrices for the two cameras Done
5. Project points back to 3d